## Usage

Using `report_ocurrences` will analyze the most advanced point of the study repo and record the ocurrences of the models pattern for each of the apps inside `study_apps` list.

Reports are unique under the (commit, app_name) tuple

(59, {'patterns.OcurrenceReport': 59})

In [2]:
import arrow
from datetime import datetime
from typing import List

from apps.patterns.git import repo_at
from apps.patterns.models import OcurrenceReport
from apps.patterns.analyzer import analyze_repo_with_pattern

def to_arrow(in_datetime: datetime, tz: str = "America/Santiago") -> arrow.Arrow:
    return arrow.Arrow(
        year=in_datetime.year,
        month=in_datetime.month,
        day=in_datetime.day,
        hour=in_datetime.hour,
        minute=in_datetime.minute,
        second=in_datetime.second,
        tzinfo=tz
    )

def report_current_ocurrences(study_app: str, ignore_paths: List[str] = []):
    with repo_at(commit=None, force_delete=False, always_pull=True) as repo:
        pattern = f"apps.{study_app}.models"
        hits = analyze_repo_with_pattern(pattern=pattern, ignore_paths=ignore_paths)
        
        commit = repo.head.commit
        payload = dict(
            app_name=study_app,
            ocurrences=hits,
            commit=commit.hexsha,
            commited_epoch=to_arrow(commit.committed_datetime).timestamp,
        )

        try:
            report = OcurrenceReport.objects.filter(commit=payload.get("commit"), app_name=payload.get("app_name")).get()
        except OcurrenceReport.DoesNotExist:
            report = OcurrenceReport.objects.create(**payload)

        return report

study_apps=dict(
    #catalog=dict(ignore_paths=["apps/catalog", "apps/promotions", "apps/stores"]),
    orders=dict(ignore_paths=["apps/orders"])
)

for app_name in study_apps:
    app_config = study_apps[app_name]
    report_current_ocurrences(study_app=app_name, **app_config)
    

In [3]:
from apps.patterns.models import OcurrenceReport
rep = OcurrenceReport.objects.filter(app_name="orders").order_by('commited_epoch').last()

print(rep.total)

655
